# Python Debugging Interview — All-in-One
Part 1: Exercises (buggy, neutral headers)
Part 2: Solutions (for interviewer reference)

**Instructions:** Run each exercise cell to see the error or incorrect behavior, then debug and fix it. Do not scroll to Part 2 until review time.

## Part 1: Exercises — Warm-up

### Exercise 1

In [ ]:
def greet(name):
print("Hello,", name)   # <- indentation issue

greet("Ada")

### Exercise 2

In [ ]:
def add_numbers(a, b):
    return a + c   # <- undefined variable

print(add_numbers(2, 3))

### Exercise 3

In [ ]:
age = "25"
print(age + 5)   # <- type mismatch

### Exercise 4

In [ ]:
def is_even(n):
    return n % 2 == 1   # <- logic bug

print("2 is even?", is_even(2))
print("3 is even?", is_even(3))

### Exercise 5

In [ ]:
def append_item(item, items=[]):
    items.append(item)
    return items

print("First call:", append_item("a"))
print("Second call:", append_item("b"))  # <- shared list surprise

## Part 1: Exercises — Bonus

### Exercise 6

In [ ]:
def read_config(path):
    f = open(path)            # may leak if parse() raises
    data = parse(f.read())    # parse is undefined here
    f.close()
    return data

try:
    read_config("config.txt")
except Exception as e:
    print(type(e).__name__, e)

### Exercise 7

In [ ]:
import csv

def save_rows(path, rows):
    f = open(path, "w")  # missing newline/encoding/context manager
    writer = csv.writer(f)
    for r in rows:
        writer.writerow(r)
    f.close()

save_rows("out.csv", [["naïve", "café"], ["a", "b"]])
print("Wrote out.csv — check for blank lines/encoding issues.")

### Exercise 8

In [ ]:
import asyncio, time

async def fetch(i):
    time.sleep(0.1)  # blocks event loop
    return i

async def main():
    tasks = [fetch(i) for i in range(5)]
    results = asyncio.gather(*tasks)  # not awaited
    print("Finished? (tasks may not have run)")
    return results

try:
    asyncio.run(main())
except Exception as e:
    print(type(e).__name__, e)

### Exercise 9

In [ ]:
import threading

counter = 0

def work(n):
    global counter
    for _ in range(n):
        counter += 1  # data race

threads = [threading.Thread(target=work, args=(100_000,)) for _ in range(4)]
for t in threads: t.start()
for t in threads: t.join()
print("Final counter:", counter, "Expected:", 400_000)

### Exercise 10

In [ ]:
import sqlite3
conn = sqlite3.connect(":memory:")
cur = conn.cursor()
cur.execute("CREATE TABLE users (id INT, name TEXT)")
cur.execute("INSERT INTO users VALUES (1, 'O\'Malley')")

username = "O'Malley"
query = f"SELECT id, name FROM users WHERE name = '{username}'"
print("Running:", query)  # breaks on quotes and is injectable
try:
    cur.execute(query)
    print(cur.fetchone())
except Exception as e:
    print(type(e).__name__, e)

### Exercise 11

In [ ]:
from datetime import datetime, timedelta

start = datetime.now()              # naive
end = start + timedelta(hours=24)
print("Seconds diff:", (end - start).total_seconds())  # assumes 86400

### Exercise 12

In [ ]:
def process(lines_iter):
    return [s.strip() for s in lines_iter if s.strip()]

data = iter(["a\n", "\n", "b\n"])
first = process(data)
second = process(data)  # empty: iterator consumed
print("first:", first, "second:", second)

### Exercise 13

In [ ]:
from dataclasses import dataclass

@dataclass
class Cart:
    items: list = []   # shared across instances

c1 = Cart()
c2 = Cart()
c1.items.append("x")
print("c2.items:", c2.items)  # unexpected shared list

### Exercise 14

In [ ]:
import os, logging
API_KEY = "sk_live_example_secret_value"  # pretend secret

def charge(amount_pennies):
    try:
        raise TimeoutError("simulated")
    except Exception as e:
        logging.error("Charge failed for key=%s amount=%s err=%r", API_KEY, amount_pennies, e)
        return False

print("Result:", charge(1234))

## Part 2: Solutions

### Solution 1

In [ ]:
def greet(name):
    print("Hello,", name)

greet("Ada")

### Solution 2

In [ ]:
def add_numbers(a, b):
    return a + b

print(add_numbers(2, 3))

### Solution 3

In [ ]:
age = "25"
print(int(age) + 5)

### Solution 4

In [ ]:
def is_even(n):
    return n % 2 == 0

print("2 is even?", is_even(2))
print("3 is even?", is_even(3))

### Solution 5

In [ ]:
def append_item(item, items=None):
    if items is None:
        items = []
    items.append(item)
    return items

print("First call:", append_item("a"))
print("Second call:", append_item("b"))

### Solution 6

In [ ]:
def read_config(path):
    # Use context manager; mock parse step
    with open(path, "w", encoding="utf-8") as f:
        f.write("key=value")
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()
    return {"raw": content}

print(read_config("config.txt"))

### Solution 7

In [ ]:
import csv
def save_rows(path, rows):
    with open(path, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerows(rows)

save_rows("out.csv", [["naïve", "café"], ["a", "b"]])
print("Wrote out.csv (UTF-8, correct newlines).")

### Solution 8

In [ ]:
import asyncio

async def fetch(i):
    await asyncio.sleep(0.1)
    return i

async def main():
    tasks = [fetch(i) for i in range(5)]
    results = await asyncio.gather(*tasks)
    print("Results:", results)

asyncio.run(main())

### Solution 9

In [ ]:
import threading

counter = 0
lock = threading.Lock()

def work(n):
    global counter
    for _ in range(n):
        with lock:
            counter += 1

threads = [threading.Thread(target=work, args=(100_000,)) for _ in range(4)]
for t in threads: t.start()
for t in threads: t.join()
print("Final counter:", counter)

### Solution 10

In [ ]:
import sqlite3
with sqlite3.connect(":memory:") as conn:
    cur = conn.cursor()
    cur.execute("CREATE TABLE users (id INT, name TEXT)")
    cur.execute("INSERT INTO users VALUES (?, ?)", (1, "O'Malley"))
    username = "O'Malley"
    cur.execute("SELECT id, name FROM users WHERE name = ?", (username,))
    print(cur.fetchone())

### Solution 11

In [ ]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

tz = ZoneInfo("Europe/London")
start = datetime.now(tz)
end = start + timedelta(hours=24)
print("Aware diff seconds:", (end - start).total_seconds())

### Solution 12

In [ ]:
def process(lines):
    return [s.strip() for s in lines if s.strip()]

data_source = ["a\n", "\n", "b\n"]
first = process(data_source)
second = process(data_source)  # works since we pass a reusable sequence
print("first:", first, "second:", second)

### Solution 13

In [ ]:
from dataclasses import dataclass, field

@dataclass
class Cart:
    items: list = field(default_factory=list)

c1 = Cart()
c2 = Cart()
c1.items.append("x")
print("c2.items:", c2.items)

### Solution 14

In [ ]:
import logging

def charge(amount_pennies):
    try:
        raise TimeoutError("simulated")
    except TimeoutError as e:
        # No secrets in logs; catch specific exception
        logging.error("Charge failed amount=%s err=%s", amount_pennies, e)
        return False

print("Result:", charge(1234))